In [ ]:
import numpy as np
import cv2, time

cv2.namedWindow("Image Feed")
cv2.moveWindow("Image Feed", 159, -25)

cap = cv2.VideoCapture(0)

# setup Cam

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS,40)

prev_frame_time = time.time()

cal_image_count = 0
frame_count = 0
while True:
    ret, frame = cap.read()
    
    #process
    frame_count +=1
    
    if frame_count == 30:
        cv2.imwrite("cal_image_" + str(cal_image_count) + ".jpg", frame)
        cal_image_count +=1
        frame_count = 0
        
    #calculate fps and display
    new_frame_time = time.time()
    fps = 1/(new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(frame, "FPS " + str(int(fps)), (10, 40), cv2.FONT_HERSHEY_PLAIN, 3, (100, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow("Image Feed", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"): break
cap.release()
cap.destroyAllWindows()

In [ ]:
import numpy as np

import cv2

import glob

cb_width = 7

cb_height = 7

cb_square_size = 23.7

#termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0), (6,5,0)
cb_3D_points = np.zeros((cb_width*cb_height, 3), np.float32)
cb_3D_points[:,:2] = np.mgrid[0:cb_width, 0:cb_height].T.reshape(-1,2)*cb_square_size

#Arrays to store object points and image points from all the images.
list_cb_3d_points = [] # 3d point in real world space
list_cb_2d_img_points = [] # 2d points in image plane.

list_images = glob.glob('*.jpg')

for frame_name in list_images: 
    img = cv2.imread(frame_name)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners

    ret, corners = cv2.findChessboardCorners (gray, (7,7), None)

    #If found , add object points, image points (after refining them)

    if ret == True:

        list_cb_3d_points.append(cb_3D_points)

        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        list_cb_2d_img_points.append(corners2)

        #Draw and display the corners

        cv2.drawChessboardCorners(img, (cb_width, cb_height), corners2, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
        
cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(list_cb_3d_points, list_cb_2d_img_points, gray.shape[::-1],None, None)

print("Calibration Matrix: ")
print(mtx)
print("Disortion: ", dist)

In [ ]:

with open('camera_cal.npy', 'wb') as f:
    np.save(f, mtx)
    np.save(f, dist)

Calibration Matrix: 
[[583.52371648   0.         296.37015552]
 [  0.         627.94558759 358.30354553]
 [  0.           0.           1.        ]]
Disortion:  [[ 0.37230818 -0.4038794   0.13492957 -0.00807964  0.30939183]]
​

In [ ]:
!pip uninstall opencv-contrib-python opencv-python

In [ ]:
!pip install opencv-contrib-python==4.6.0.66

In [ ]:
import numpy as np
import cv2
import cv2.aruco as aruco

marker_size = 100
# the camera matrix and camera distortion shouldm be numpy array type not list type
camera_matrix = np.array([[583.52371648,   0.,         296.37015552], 
 [  0.,         627.94558759, 358.30354553],
 [  0.,           0.,           1.        ]])
camera_distortion = np.array([[ 0.37230818, -0.4038794,   0.13492957, -0.00807964,  0.30939183]])

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
parameters =  aruco.DetectorParameters()
detector = aruco.ArucoDetector(aruco_dict, parameters)
cap = cv2.VideoCapture(0)

camera_width = 640
camera_height = 480
camera_frame_rate = 40

cap.set(2, camera_width)
cap.set(4, camera_height)
cap.set(5, camera_frame_rate)

while True:
    
    ret, frame = cap.read()

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    corners, ids, rejected = detector.detectMarkers(gray_frame)

    if ids is not None: # detecting if any problemetic frames come into code we can use a try except statement to get rid of problematic frames
        aruco.drawDetectedMarkers(frame, corners)
        #ret = aruco.estimatePoseSingleMarkers(corners,marker_size,cameraMatrix=cameraMatrix,distCoeffs=cameraDistortion)
        ret = aruco.estimatePoseSingleMarkers(corners, marker_size,cameraMatrix= camera_matrix,distCoeffs= camera_distortion)
        rvec,tvec = ret[0][0,0,:], ret[1][0,0,:]
        aruco.drawAxis(frame, camera_matrix, camera_distortion, rvec, tvec, 100)
        tvec_str = "x=%4.0f   y=%4.0f  z=%4.0f"%(tvec[0], tvec[1], tvec[2])
        cv2.putText(frame, tvec_str, (20, 460), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)
        
        
        
    cv2.imshow('frame',frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):break

cap.release()
cv2.destroyAllWindows()

b.

In [18]:
import numpy as np
import cv2
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation

def focalMM_to_focalPixel(focalMM, pixelPitch):
    return focalMM / pixelPitch

# Camera matrix and distortion coefficients
camera_matrix = np.array([
    [583.52371648, 0., 296.37015552], 
    [0., 627.94558759, 358.30354553],
    [0., 0., 1.]
])
camera_distortion = np.array([0.37230818, -0.4038794, 0.13492957, -0.00807964, 0.30939183])

# Define the dictionary and detector parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
parameters = aruco.DetectorParameters()

# Define the 3D coordinates of the marker corners in meters
marker_size = 0.1  # size of the marker in meters
obj_points = np.array([
    [-marker_size / 2, marker_size / 2, 0],
    [ marker_size / 2, marker_size / 2, 0],
    [ marker_size / 2, -marker_size / 2, 0],
    [-marker_size / 2, -marker_size / 2, 0]
], dtype=np.float32)

# Initialize the video capture
cap = cv2.VideoCapture(0)

camera_width = 640
camera_height = 480
camera_frame_rate = 40

cap.set(cv2.CAP_PROP_FRAME_WIDTH, camera_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, camera_height)
cap.set(cv2.CAP_PROP_FPS, camera_frame_rate)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray_frame)

    if ids is not None:
        # Draw detected markers
        aruco.drawDetectedMarkers(frame, corners, ids)

        for i in range(len(ids)):
            # Estimate pose of each marker using SOLVEPNP_IPPE_SQUARE
            success, rvec, tvec = cv2.solvePnP(obj_points, corners[i][0], camera_matrix, camera_distortion, flags=cv2.SOLVEPNP_IPPE_SQUARE)

            if success:
                rvec = rvec.reshape((3, 1))  # Ensure rvec is 3x1
                tvec = tvec.reshape((3, 1))  # Ensure tvec is 3x1

                # Draw axis for each marker
                cv2.drawFrameAxes(frame, camera_matrix, camera_distortion, rvec, tvec, 0.1)

                # Convert translation from meters to millimeters
                tvec_mm = tvec * 1000  # Convert to millimeters

                # Calculate camera position
                rmat = cv2.Rodrigues(rvec)[0]
                camera_position = -np.matrix(rmat).T @ np.matrix(tvec)

                # Extract scalar values from the translation vector in mm
                tvec_str = "x={:.2f} mm y={:.2f} mm z={:.2f} mm".format(tvec_mm[0][0], tvec_mm[1][0], tvec_mm[2][0])
                cv2.putText(frame, tvec_str, (20, 460), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)

                # Extract rotation angles
                r = Rotation.from_rotvec([rvec[0][0], rvec[1][0], rvec[2][0]])
                rot = r.as_euler('xyz', degrees=True)

                tx = round(camera_position[0, 0] * 1000, 2)  # Convert to millimeters
                ty = round(camera_position[1, 0] * 1000, 2)  # Convert to millimeters
                tz = round(camera_position[2, 0] * 1000, 2)  # Convert to millimeters

                rx = round(180 - rot[0], 2)
                ry = round(rot[1], 2)
                rz = round(rot[2], 2)

                rot_str = "rx={:.2f} ry={:.2f} rz={:.2f}".format(rx, ry, rz)
                cv2.putText(frame, rot_str, (20, 420), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('frame', frame)

    # Break the loop on 'q' key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
import cv2
print(cv2.__version__)
print(dir(cv2.aruco))

4.10.0
['ArucoDetector', 'Board', 'CORNER_REFINE_APRILTAG', 'CORNER_REFINE_CONTOUR', 'CORNER_REFINE_NONE', 'CORNER_REFINE_SUBPIX', 'CharucoBoard', 'CharucoDetector', 'CharucoParameters', 'DICT_4X4_100', 'DICT_4X4_1000', 'DICT_4X4_250', 'DICT_4X4_50', 'DICT_5X5_100', 'DICT_5X5_1000', 'DICT_5X5_250', 'DICT_5X5_50', 'DICT_6X6_100', 'DICT_6X6_1000', 'DICT_6X6_250', 'DICT_6X6_50', 'DICT_7X7_100', 'DICT_7X7_1000', 'DICT_7X7_250', 'DICT_7X7_50', 'DICT_APRILTAG_16H5', 'DICT_APRILTAG_16h5', 'DICT_APRILTAG_25H9', 'DICT_APRILTAG_25h9', 'DICT_APRILTAG_36H10', 'DICT_APRILTAG_36H11', 'DICT_APRILTAG_36h10', 'DICT_APRILTAG_36h11', 'DICT_ARUCO_MIP_36H12', 'DICT_ARUCO_MIP_36h12', 'DICT_ARUCO_ORIGINAL', 'DetectorParameters', 'Dictionary', 'Dictionary_getBitsFromByteList', 'Dictionary_getByteListFromBits', 'GridBoard', 'RefineParameters', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_native', 'drawDetectedCornersCharuco', 'drawDetectedDiamonds', 'drawDetectedMar

In [ ]:
!pip install imutils

In [ ]:
#sudo nano  is nano editior
#nano is a GNU editior 
#chmod77 what does it do 
#C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12456\1244901739.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
#  frame_np = np.array(frame)
#---------------------------------------------------------------------------
#error                                     Traceback (most recent call last)
#Cell In[6], line 59
#     56 #    frame = cv2.resize(frame,(width,height))
#     58     frame_np = np.array(frame)
#---> 59     gray_img = cv2.cvtColor(frame_np,cv2.COLOR_BGR2GRAY)
#     60     ids=''
#     61     corners, ids, rejected = aruco.detectMarkers(image=gray_img,dictionary=aruco_dict,parameters=parameters)
#
#error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
#> Overload resolution failed:
#>  - src data type = object is not supported
#>  - Expected Ptr<cv::UMat> for argument 'src'

import numpy as np
import cv2
from cv2 import aruco

cap = cv2.VideoCapture(0)

dictionary = aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)

while(True):
    #Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    parameters = aruco.DetectorParameters()
    detector = aruco.ArucoDetector(dictionary, parameters)
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray)
    frame_markers = aruco.drawDetectedMarkers (frame.copy(), corners, ids)

    cv2.imshow('Aruco Markers', frame_markers)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Detect and feed to model

In [10]:
import numpy as np
import cv2
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation
import time

def focalMM_to_focalPixel(focalMM, pixelPitch):
    return focalMM / pixelPitch

# Camera matrix and distortion coefficients
camera_matrix = np.array([
    [583.52371648, 0., 296.37015552], 
    [0., 627.94558759, 358.30354553],
    [0., 0., 1.]
])
camera_distortion = np.array([0.37230818, -0.4038794, 0.13492957, -0.00807964, 0.30939183])

# Define the dictionary and detector parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
parameters = aruco.DetectorParameters()

# Define the 3D coordinates of the marker corners in meters
marker_size = 0.1  # size of the marker in meters
obj_points = np.array([
    [-marker_size / 2, marker_size / 2, 0],
    [ marker_size / 2, marker_size / 2, 0],
    [ marker_size / 2, -marker_size / 2, 0],
    [-marker_size / 2, -marker_size / 2, 0]
], dtype=np.float32)

# Initialize the video capture
cap = cv2.VideoCapture(0)

camera_width = 640
camera_height = 480
camera_frame_rate = 40

cap.set(cv2.CAP_PROP_FRAME_WIDTH, camera_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, camera_height)
cap.set(cv2.CAP_PROP_FPS, camera_frame_rate)

prev_time = time.time()
frame_count = 0
fps = 0  # Variable to store the FPS value

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray_frame)

    if ids is not None:
        # Draw detected markers
        aruco.drawDetectedMarkers(frame, corners, ids)

        for i in range(len(ids)):
            # Estimate pose of each marker using SOLVEPNP_IPPE_SQUARE
            success, rvec, tvec = cv2.solvePnP(obj_points, corners[i][0], camera_matrix, camera_distortion, flags=cv2.SOLVEPNP_IPPE_SQUARE)

            if success:
                rvec = rvec.reshape((3, 1))  # Ensure rvec is 3x1
                tvec = tvec.reshape((3, 1))  # Ensure tvec is 3x1

                # Draw axis for each marker
                cv2.drawFrameAxes(frame, camera_matrix, camera_distortion, rvec, tvec, 0.1)

                # Convert translation from meters to millimeters
                tvec_mm = tvec * 1000  # Convert to millimeters

                # Calculate camera position
                rmat = cv2.Rodrigues(rvec)[0]
                camera_position = -np.matrix(rmat).T @ np.matrix(tvec)

                # Extract scalar values from the translation vector in mm
                tvec_str = "x={:.2f} mm y={:.2f} mm z={:.2f} mm".format(tvec_mm[0][0], tvec_mm[1][0], tvec_mm[2][0])
                cv2.putText(frame, tvec_str, (20, 460), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)

                # Extract rotation angles
                r = Rotation.from_rotvec([rvec[0][0], rvec[1][0], rvec[2][0]])
                rot = r.as_euler('xyz', degrees=True)

                tx = round(camera_position[0, 0] * 1000, 2)  # Convert to millimeters
                ty = round(camera_position[1, 0] * 1000, 2)  # Convert to millimeters
                tz = round(camera_position[2, 0] * 1000, 2)  # Convert to millimeters

                rx = round(180 - rot[0], 2)
                ry = round(rot[1], 2)
                rz = round(rot[2], 2)

                rot_str = "rx={:.2f} ry={:.2f} rz={:.2f}".format(rx, ry, rz)
                cv2.putText(frame, rot_str, (20, 420), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2, cv2.LINE_AA)

    # Calculate and display FPS every 10 frames
    frame_count += 1
    if frame_count >= 10:
        current_time = time.time()
        elapsed_time = current_time - prev_time
        fps = frame_count / elapsed_time
        frame_count = 0
        prev_time = current_time

    fps_str = "FPS: {:.2f}".format(fps)
    cv2.putText(frame, fps_str, (20, 380), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('frame', frame)

    # Break the loop on 'q' key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [12]:
import numpy as np
import cv2
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation

def focalMM_to_focalPixel(focalMM, pixelPitch):
    return focalMM / pixelPitch

# Camera matrix and distortion coefficients
camera_matrix = np.array([
    [583.52371648, 0., 296.37015552], 
    [0., 627.94558759, 358.30354553],
    [0., 0., 1.]
])
camera_distortion = np.array([0.37230818, -0.4038794, 0.13492957, -0.00807964, 0.30939183])

# Define the dictionary and detector parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
parameters = aruco.DetectorParameters()

# Define the 3D coordinates of the marker corners in meters
marker_size = 0.1  # size of the marker in meters
obj_points = np.array([
    [-marker_size / 2, marker_size / 2, 0],
    [ marker_size / 2, marker_size / 2, 0],
    [ marker_size / 2, -marker_size / 2, 0],
    [-marker_size / 2, -marker_size / 2, 0]
], dtype=np.float32)

# Read the image
image_path = '5e9449ce-8fc2-4268-a3e7-804d97cad046.jpg'  # Replace with your image path
frame = cv2.imread(image_path)

if frame is not None:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray_frame)

    if ids is not None:
        # Draw detected markers
        aruco.drawDetectedMarkers(frame, corners, ids)

        for i in range(len(ids)):
            # Estimate pose of each marker using SOLVEPNP_IPPE_SQUARE
            success, rvec, tvec = cv2.solvePnP(obj_points, corners[i][0], camera_matrix, camera_distortion, flags=cv2.SOLVEPNP_IPPE_SQUARE)

            if success:
                rvec = rvec.reshape((3, 1))  # Ensure rvec is 3x1
                tvec = tvec.reshape((3, 1))  # Ensure tvec is 3x1

                # Draw axis for each marker
                cv2.drawFrameAxes(frame, camera_matrix, camera_distortion, rvec, tvec, 0.1)

                # Convert translation from meters to millimeters
                tvec_mm = tvec * 1000  # Convert to millimeters

                # Calculate camera position
                rmat = cv2.Rodrigues(rvec)[0]
                camera_position = -np.matrix(rmat).T @ np.matrix(tvec)

                # Extract scalar values from the translation vector in mm
                tvec_str = "x={:.2f} mm y={:.2f} mm z={:.2f} mm".format(tvec_mm[0][0], tvec_mm[1][0], tvec_mm[2][0])
                
                # Extract rotation angles
                r = Rotation.from_rotvec([rvec[0][0], rvec[1][0], rvec[2][0]])
                rot = r.as_euler('xyz', degrees=True)

                tx = round(camera_position[0, 0] * 1000, 2)  # Convert to millimeters
                ty = round(camera_position[1, 0] * 1000, 2)  # Convert to millimeters
                tz = round(camera_position[2, 0] * 1000, 2)  # Convert to millimeters

                rx = round(180 - rot[0], 2)
                ry = round(rot[1], 2)
                rz = round(rot[2], 2)

                rot_str = "rx={:.2f} ry={:.2f} rz={:.2f}".format(rx, ry, rz)

                # Display the text at the top of the image
                cv2.putText(frame, tvec_str, (20, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, rot_str, (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('frame', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print(f"Failed to load image at {image_path}")

# __3.Execute Image with ARUCO /\ Object detection tflite__

In [8]:
import numpy as np
import cv2
import tensorflow as tf
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation

def load_labels(label_path):
    with open(label_path, 'r') as f:
        return [line.strip() for line in f.readlines()]

def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def get_output_tensor(interpreter, index):
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor

def detect_objects(interpreter, image, threshold):
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    scores = get_output_tensor(interpreter, 0)
    boxes = get_output_tensor(interpreter, 1)

    results = []
    for i in range(scores.shape[0]):
        if scores[i] >= threshold:
            result = {
                'bounding_box': boxes[i],
                'score': scores[i]
            }
            results.append(result)
    return results

def draw_bounding_boxes(image, results):
    height, width, _ = image.shape
    for result in results:
        ymin, xmin, ymax, xmax = result['bounding_box']
        ymin = int(max(1, ymin * height))
        xmin = int(max(1, xmin * width))
        ymax = int(min(height, ymax * height))
        xmax = int(min(width, xmax * width))

        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        label = f"{result['score']:.2f}"
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

def main():
    model_path = "detect.tflite"
    label_path = "labels.txt"
    threshold = 0.5
    image_path = "0a0097e5-WhatsApp_Image_2024-07-28_at_4.36.09_PM_2.jpeg"  # Update this path to your image

    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    labels = load_labels(label_path)

    # Load image
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Failed to load image at {image_path}")
        return

    # ArUco marker detection setup
    camera_matrix = np.array([
        [583.52371648, 0., 296.37015552], 
        [0., 627.94558759, 358.30354553],
        [0., 0., 1.]
    ])
    camera_distortion = np.array([0.37230818, -0.4038794, 0.13492957, -0.00807964, 0.30939183])
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
    parameters = aruco.DetectorParameters()
    marker_size = 0.1  # size of the marker in meters
    obj_points = np.array([
        [-marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, -marker_size / 2, 0],
        [-marker_size / 2, -marker_size / 2, 0]
    ], dtype=np.float32)

    # ArUco marker detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray_frame)

    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        for i in range(len(ids)):
            success, rvec, tvec = cv2.solvePnP(obj_points, corners[i][0], camera_matrix, camera_distortion, flags=cv2.SOLVEPNP_IPPE_SQUARE)
            if success:
                cv2.drawFrameAxes(frame, camera_matrix, camera_distortion, rvec, tvec, 0.1)
                tvec_mm = tvec * 1000
                rmat = cv2.Rodrigues(rvec)[0]
                camera_position = -np.matrix(rmat).T @ np.matrix(tvec)
                tvec_str = "x={:.2f} mm y={:.2f} mm z={:.2f} mm".format(tvec_mm[0][0], tvec_mm[1][0], tvec_mm[2][0])
                r = Rotation.from_rotvec([rvec[0][0], rvec[1][0], rvec[2][0]])
                rot = r.as_euler('xyz', degrees=True)
                rx = round(180 - rot[0], 2)
                ry = round(rot[1], 2)
                rz = round(rot[2], 2)
                rot_str = "rx={:.2f} ry={:.2f} rz={:.2f}".format(rx, ry, rz)
                cv2.putText(frame, tvec_str, (20, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, rot_str, (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2, cv2.LINE_AA)

    # Preprocessing for TFLite model input
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (640, 640))  # Update size based on model input
    input_image = np.expand_dims(image_resized / 255.0, axis=0).astype(np.float32)  # Normalization

    # Perform object detection
    results = detect_objects(interpreter, input_image, threshold)
    draw_bounding_boxes(frame, results)

    # Display the frame with bounding boxes and ArUco data
    cv2.imshow('Object Detection', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

# __4.Execute Image with ARUCO /\ Object detection tflite /\ Distance__

In [9]:
import numpy as np
import cv2
import tensorflow as tf
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation

def load_labels(label_path):
    with open(label_path, 'r') as f:
        return [line.strip() for line in f.readlines()]

def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def get_output_tensor(interpreter, index):
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor

def detect_objects(interpreter, image, threshold):
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    scores = get_output_tensor(interpreter, 0)
    boxes = get_output_tensor(interpreter, 1)

    results = []
    for i in range(scores.shape[0]):
        if scores[i] >= threshold:
            result = {
                'bounding_box': boxes[i],
                'score': scores[i]
            }
            results.append(result)
    return results

def draw_bounding_boxes(image, results):
    height, width, _ = image.shape
    centers = []
    for result in results:
        ymin, xmin, ymax, xmax = result['bounding_box']
        ymin = int(max(1, ymin * height))
        xmin = int(max(1, xmin * width))
        ymax = int(min(height, ymax * height))
        xmax = int(min(width, xmax * width))

        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        label = f"{result['score']:.2f}"
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Calculate the center of the bounding box
        center_x = (xmin + xmax) // 2
        center_y = (ymin + ymax) // 2
        centers.append((center_x, center_y))

    return centers

def main():
    model_path = "detect.tflite"
    label_path = "labels.txt"
    threshold = 0.5
    image_path = "0a0097e5-WhatsApp_Image_2024-07-28_at_4.36.09_PM_2.jpeg"  # Update this path to your image

    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    labels = load_labels(label_path)

    # Load image
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Failed to load image at {image_path}")
        return

    # ArUco marker detection setup
    camera_matrix = np.array([
        [583.52371648, 0., 296.37015552], 
        [0., 627.94558759, 358.30354553],
        [0., 0., 1.]
    ])
    camera_distortion = np.array([0.37230818, -0.4038794, 0.13492957, -0.00807964, 0.30939183])
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
    parameters = aruco.DetectorParameters()
    marker_size = 0.1  # size of the marker in meters
    obj_points = np.array([
        [-marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, -marker_size / 2, 0],
        [-marker_size / 2, -marker_size / 2, 0]
    ], dtype=np.float32)

    # ArUco marker detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray_frame)

    aruco_center = None
    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        for i in range(len(ids)):
            success, rvec, tvec = cv2.solvePnP(obj_points, corners[i][0], camera_matrix, camera_distortion, flags=cv2.SOLVEPNP_IPPE_SQUARE)
            if success:
                cv2.drawFrameAxes(frame, camera_matrix, camera_distortion, rvec, tvec, 0.1)
                tvec_mm = tvec * 1000
                rmat = cv2.Rodrigues(rvec)[0]
                camera_position = -np.matrix(rmat).T @ np.matrix(tvec)
                tvec_str = "x={:.2f} mm y={:.2f} mm z={:.2f} mm".format(tvec_mm[0][0], tvec_mm[1][0], tvec_mm[2][0])
                r = Rotation.from_rotvec([rvec[0][0], rvec[1][0], rvec[2][0]])
                rot = r.as_euler('xyz', degrees=True)
                rx = round(180 - rot[0], 2)
                ry = round(rot[1], 2)
                rz = round(rot[2], 2)
                rot_str = "rx={:.2f} ry={:.2f} rz={:.2f}".format(rx, ry, rz)
                cv2.putText(frame, tvec_str, (20, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, rot_str, (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2, cv2.LINE_AA)

                # Calculate the center of the ArUco marker
                aruco_center = np.mean(corners[i][0], axis=0).astype(int)

    # Preprocessing for TFLite model input
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (640, 640))  # Update size based on model input
    input_image = np.expand_dims(image_resized / 255.0, axis=0).astype(np.float32)  # Normalization

    # Perform object detection
    results = detect_objects(interpreter, input_image, threshold)
    box_centers = draw_bounding_boxes(frame, results)

    # Draw lines from the ArUco marker center to each bounding box center
    if aruco_center is not None:
        for center in box_centers:
            cv2.line(frame, tuple(aruco_center), center, (0, 255, 0), 2)

    # Display the frame with bounding boxes and ArUco data
    cv2.imshow('Object Detection', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

## __5.Execute With ARUCO /\ Object detection tflite /\ Distance /\ Export__

In [6]:
import numpy as np
import cv2
import tensorflow as tf
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation

def load_labels(label_path):
    with open(label_path, 'r') as f:
        return [line.strip() for line in f.readlines()]

def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def get_output_tensor(interpreter, index):
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor

def detect_objects(interpreter, image, threshold):
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    scores = get_output_tensor(interpreter, 0)
    boxes = get_output_tensor(interpreter, 1)

    results = []
    for i in range(scores.shape[0]):
        if scores[i] >= threshold:
            result = {
                'bounding_box': boxes[i],
                'score': scores[i]
            }
            results.append(result)
    return results

def draw_bounding_boxes(image, results):
    height, width, _ = image.shape
    centers = []
    for result in results:
        ymin, xmin, ymax, xmax = result['bounding_box']
        ymin = int(max(1, ymin * height))
        xmin = int(max(1, xmin * width))
        ymax = int(min(height, ymax * height))
        xmax = int(min(width, xmax * width))

        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 255), 2)  # Yellow color for bounding boxes
        label = f"{result['score']:.2f}"
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)  # Yellow color for confidence values

        # Calculate the center of the bounding box
        center_x = (xmin + xmax) // 2
        center_y = (ymin + ymax) // 2
        centers.append((center_x, center_y))

    return centers

def crop_and_save_leaves(image, results, output_dir='cropped_leaves'):
    height, width, _ = image.shape
    leaf_images = []
    for idx, result in enumerate(results):
        ymin, xmin, ymax, xmax = result['bounding_box']
        ymin = int(max(1, ymin * height))
        xmin = int(max(1, xmin * width))
        ymax = int(min(height, ymax * height))
        xmax = int(min(width, xmax * width))

        # Crop the bounding box region from the image
        leaf_image = image[ymin:ymax, xmin:xmax]
        leaf_images.append(leaf_image)

        # Save the cropped leaf image
        output_path = f"{output_dir}/leaf_{idx}.jpg"
        cv2.imwrite(output_path, leaf_image)
    
    return leaf_images

def main():
    model_path = "detect.tflite"
    label_path = "labels.txt"
    threshold = 0.5
    image_path = "0a0097e5-WhatsApp_Image_2024-07-28_at_4.36.09_PM_2.jpeg"  # Update this path to your image

    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    labels = load_labels(label_path)

    # Load image
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Failed to load image at {image_path}")
        return

    # ArUco marker detection setup
    camera_matrix = np.array([
        [583.52371648, 0., 296.37015552], 
        [0., 627.94558759, 358.30354553],
        [0., 0., 1.]
    ])
    camera_distortion = np.array([0.37230818, -0.4038794, 0.13492957, -0.00807964, 0.30939183])
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
    parameters = aruco.DetectorParameters()
    marker_size = 0.1  # size of the marker in meters
    obj_points = np.array([
        [-marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, -marker_size / 2, 0],
        [-marker_size / 2, -marker_size / 2, 0]
    ], dtype=np.float32)

    # ArUco marker detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray_frame)

    aruco_center = None
    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        for i in range(len(ids)):
            success, rvec, tvec = cv2.solvePnP(obj_points, corners[i][0], camera_matrix, camera_distortion, flags=cv2.SOLVEPNP_IPPE_SQUARE)
            if success:
                cv2.drawFrameAxes(frame, camera_matrix, camera_distortion, rvec, tvec, 0.1)
                tvec_mm = tvec * 1000
                rmat = cv2.Rodrigues(rvec)[0]
                camera_position = -np.matrix(rmat).T @ np.matrix(tvec)
                tvec_str = "x={:.2f} mm y={:.2f} mm z={:.2f} mm".format(tvec_mm[0][0], tvec_mm[1][0], tvec_mm[2][0])
                r = Rotation.from_rotvec([rvec[0][0], rvec[1][0], rvec[2][0]])
                rot = r.as_euler('xyz', degrees=True)
                rx = round(180 - rot[0], 2)
                ry = round(rot[1], 2)
                rz = round(rot[2], 2)
                rot_str = "rx={:.2f} ry={:.2f} rz={:.2f}".format(rx, ry, rz)
                cv2.putText(frame, tvec_str, (20, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, rot_str, (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2, cv2.LINE_AA)

                # Calculate the center of the ArUco marker
                aruco_center = np.mean(corners[i][0], axis=0).astype(int)

    # Preprocessing for TFLite model input
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (640, 640))  # Update size based on model input
    input_image = np.expand_dims(image_resized / 255.0, axis=0).astype(np.float32)  # Normalization

    # Perform object detection
    results = detect_objects(interpreter, input_image, threshold)
    box_centers = draw_bounding_boxes(frame, results)

    # Crop and save leaf images
    crop_and_save_leaves(frame, results)

    # Draw lines from the ArUco marker center to each bounding box center
    if aruco_center is not None:
        for center in box_centers:
            cv2.line(frame, tuple(aruco_center), center, (0, 255, 0), 2)

    # Display the frame with bounding boxes and ArUco data
    cv2.imshow('Object Detection', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [13]:
import numpy as np
import cv2
import tensorflow as tf
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation

def load_labels(label_path):
    with open(label_path, 'r') as f:
        return [line.strip() for line in f.readlines()]

def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def get_output_tensor(interpreter, index):
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor

def detect_objects(interpreter, image, threshold):
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    scores = get_output_tensor(interpreter, 0)
    boxes = get_output_tensor(interpreter, 1)
    classes = get_output_tensor(interpreter, 2)  # Assuming this is where the classes are stored

    print(f"Scores shape: {scores.shape}")
    print(f"Boxes shape: {boxes.shape}")
    print(f"Classes shape: {classes.shape}")

    results = []
    for i in range(scores.shape[0]):
        if scores[i] >= threshold:
            if len(classes.shape) > 0:  # Check if 'classes' is an array
                label_id = int(classes[i])
            else:
                label_id = int(classes)  # If scalar, use it directly

            result = {
                'bounding_box': boxes[i],
                'score': scores[i],
                'label_id': label_id
            }
            results.append(result)

    print(f"Detected Objects: {results}")
    return results

def draw_bounding_boxes(image, results, labels):
    height, width, _ = image.shape
    centers = []
    for result in results:
        ymin, xmin, ymax, xmax = result['bounding_box']
        ymin = int(max(1, ymin * height))
        xmin = int(max(1, xmin * width))
        ymax = int(min(height, ymax * height))
        xmax = int(min(width, xmax * width))

        # Draw the bounding box
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 255), 2)

        # Draw the label and confidence score
        label = f"{labels[result['label_id']]}: {result['score']:.2f}"
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        # Calculate the center of the bounding box
        center_x = (xmin + xmax) // 2
        center_y = (ymin + ymax) // 2
        centers.append((center_x, center_y))

    return centers

def crop_and_save_leaves(image, results, output_dir='cropped_leaves'):
    height, width, _ = image.shape
    leaf_images = []
    for idx, result in enumerate(results):
        ymin, xmin, ymax, xmax = result['bounding_box']
        ymin = int(max(1, ymin * height))
        xmin = int(max(1, xmin * width))
        ymax = int(min(height, ymax * height))
        xmax = int(min(width, xmax * width))

        leaf_image = image[ymin:ymax, xmin:xmax]
        leaf_images.append(leaf_image)

        output_path = f"{output_dir}/leaf_{idx}.jpg"
        cv2.imwrite(output_path, leaf_image)
    
    return leaf_images

def main():
    model_path = "detect.tflite"
    label_path = "labels.txt"
    threshold = 0.5
    image_path = "0a0097e5-WhatsApp_Image_2024-07-28_at_4.36.09_PM_2.jpeg"

    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    labels = load_labels(label_path)

    # Load image
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Failed to load image at {image_path}")
        return

    # ArUco marker detection setup
    camera_matrix = np.array([
        [583.52371648, 0., 296.37015552], 
        [0., 627.94558759, 358.30354553],
        [0., 0., 1.]
    ])
    camera_distortion = np.array([0.37230818, -0.4038794, 0.13492957, -0.00807964, 0.30939183])
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
    parameters = aruco.DetectorParameters()
    marker_size = 0.1  # size of the marker in meters
    obj_points = np.array([
        [-marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, marker_size / 2, 0],
        [marker_size / 2, -marker_size / 2, 0],
        [-marker_size / 2, -marker_size / 2, 0]
    ], dtype=np.float32)

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray_frame)

    aruco_center = None
    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        for i in range(len(ids)):
            success, rvec, tvec = cv2.solvePnP(obj_points, corners[i][0], camera_matrix, camera_distortion, flags=cv2.SOLVEPNP_IPPE_SQUARE)
            if success:
                cv2.drawFrameAxes(frame, camera_matrix, camera_distortion, rvec, tvec, 0.1)
                tvec_mm = tvec * 1000
                rmat = cv2.Rodrigues(rvec)[0]
                camera_position = -np.matrix(rmat).T @ np.matrix(tvec)
                tvec_str = "x={:.2f} mm y={:.2f} mm z={:.2f} mm".format(tvec_mm[0][0], tvec_mm[1][0], tvec_mm[2][0])
                r = Rotation.from_rotvec([rvec[0][0], rvec[1][0], rvec[2][0]])
                rot = r.as_euler('xyz', degrees=True)
                rx = round(180 - rot[0], 2)
                ry = round(rot[1], 2)
                rz = round(rot[2], 2)
                rot_str = "rx={:.2f} ry={:.2f} rz={:.2f}".format(rx, ry, rz)
                cv2.putText(frame, tvec_str, (20, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, rot_str, (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2, cv2.LINE_AA)

                aruco_center = np.mean(corners[i][0], axis=0).astype(int)

    # Preprocessing for TFLite model input
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (640, 640))
    input_image = np.expand_dims(image_resized / 255.0, axis=0).astype(np.float32)

    # Perform object detection
    results = detect_objects(interpreter, input_image, threshold)
    box_centers = draw_bounding_boxes(frame, results, labels)

    crop_and_save_leaves(frame, results)

    if aruco_center is not None:
        for center in box_centers:
            cv2.line(frame, tuple(aruco_center), center, (0, 255, 0), 2)

    cv2.imshow('Object Detection', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Scores shape: (10,)
Boxes shape: (10, 4)
Classes shape: ()
Detected Objects: [{'bounding_box': array([0.29976922, 0.42708132, 0.39245367, 0.49433598], dtype=float32), 'score': 0.99456084, 'label_id': 10}, {'bounding_box': array([0.21030797, 0.40645975, 0.26754925, 0.47419035], dtype=float32), 'score': 0.9932978, 'label_id': 10}, {'bounding_box': array([0.3966902 , 0.15968618, 0.46919703, 0.2135821 ], dtype=float32), 'score': 0.98756444, 'label_id': 10}, {'bounding_box': array([0.26730022, 0.47824264, 0.35140058, 0.55542517], dtype=float32), 'score': 0.98134744, 'label_id': 10}, {'bounding_box': array([0.4784892 , 0.19943172, 0.625328  , 0.28374678], dtype=float32), 'score': 0.97706556, 'label_id': 10}, {'bounding_box': array([0.4112168 , 0.35256162, 0.4603334 , 0.40973184], dtype=float32), 'score': 0.9680633, 'label_id': 10}, {'bounding_box': array([0.32386237, 0.21596752, 0.36344522, 0.29107404], dtype=float32), 'score': 0.9433292, 'label_id': 10}, {'bounding_box': array([0.41253722, 

IndexError: list index out of range